# Assignment 2
Group names

1. Mehmet Güray Güler
2. İsmail Sevim

#### 1. Get the data into a Pandas DataFrame.


In [ ]:
import csv
import requests
import pandas as pd
import openpyxl

link = "https://docs.google.com/spreadsheets/d/1isOVWu-C6UqGy7Kg7sabE6O_Uq1AsMgK/edit#gid=215309859"

# Extract the gid from the URL
gid = link.split("#gid=")[1]

# Use the gid to download the sheet as a CSV file
link = f"https://docs.google.com/spreadsheets/d/1isOVWu-C6UqGy7Kg7sabE6O_Uq1AsMgK/export?gid={gid}&format=csv"
response = requests.get(link)


# Check if the request was successful
if response.status_code == 200:
    # If the request was successful, the data is stored in the response content
    data = response.content
else:
    # If the request was not successful, print an error message
    print("Failed to fetch data")

# Parse the data and extract the year and price for each car
reader = csv.reader(data.decode("utf-8").splitlines())

# Convert the reader object to a list
csv_data = list(reader)

# Get the column names from the first row of the CSV file
column_names = csv_data[0]

# Remove the first row from the data
data = csv_data[1:]

# Create the DataFrame, specifying the column names
df = pd.DataFrame(data, columns=column_names)

df['Price'] = pd.to_numeric(df['Price'])
df['Year'] = pd.to_numeric(df['Year'])
df['Km'] = pd.to_numeric(df['Km'])
df['Power'] = pd.to_numeric(df['Power'])
df['Engine'] = pd.to_numeric(df['Engine'])


## 2. Data Cleaning

Remove the cars where there exists the same car with the same values for all the columns except the price.


In [ ]:
# Create a list of columns to keep
columns_to_keep = df.columns.difference(['Price']).to_list()

# Create a copy of the DataFrame
df_sorted = df.copy()

# Sort the copy by price
df_sorted.sort_values('Price', ascending=True, inplace=True)

# Drop duplicates
df_sorted = df_sorted.drop_duplicates(subset=columns_to_keep, keep='first')

# Filter the original DataFrame
df = df[df.isin(df_sorted)].dropna(how='all')

# Save the DataFrame to an Excel file
df.to_excel("Car_Sales_Data_Analysis.xlsx", index=False)


#### 3. Find the average price of cars for each years and return it as a dictionary.


In [ ]:
# Group the data by the "Year" column and compute the average price for each group
price_by_year = df.groupby('Year')['Price'].mean()

# Convert the index to integers
price_by_year.index = price_by_year.index.astype(int)

# Convert the Series to a dictionary
price_by_year_dict = price_by_year.to_dict()

print(price_by_year_dict)


#### 4. Find the cars with the missing prices and replace them with the average prices of the years they belong to.


In [ ]:
# Find the rows where the 'Price' column is null
null_rows = df[df['Price'].isnull()]

# Iterate through the rows and fill in the null values with the average price for the corresponding year
for i, row in null_rows.iterrows():
    year = row['Year']
    avg_price = price_by_year_dict[year]
    df.at[i, 'Price'] = avg_price


#### 5. Find the standard deviation of the prices of each brand and return it as a dictionary.


In [ ]:
# Group the data by the 'Brand' column and compute the standard deviation of the prices for each group
# price_std_by_brand = df.groupby('Brand')['Price'].std() # This is the easy way to do it

# Calculate standard deviation manually
price_std_by_brand = df.groupby('Brand')['Price'].apply(lambda group: (sum((x - group.mean())**2 for x in group) / (len(group) - 1))**0.5)

# Convert the Series to a dictionary
price_std_by_brand_dict = price_std_by_brand.to_dict()

print(price_std_by_brand_dict)


#### 5. Find the average price of each city and return it as a dictionary.


In [ ]:
# Group the data by the 'City' column and compute the average price for each group
price_avg_by_city = df.groupby('City')['Price'].mean()

# Convert the Series to a dictionary
price_avg_by_city_dict = price_avg_by_city.to_dict()

print(price_avg_by_city_dict)


### 6. Find the average price of each brand and return it as a dictionary.


In [ ]:
# Group the data by the 'City' column and compute the average price for each group
price_avg_by_city = df.groupby('City')['Price'].mean()

# Convert the Series to a dictionary
price_avg_by_city_dict = price_avg_by_city.to_dict()

print(price_avg_by_city_dict)


### 7. Find the average price of the Mercedes-Benz cars that has automatic transmission in İstanbul.


In [ ]:
# Select the rows that have 'Mercedes-Benz' in the 'Brand' column and 'Automatic' in the 'Transmission' column
selected_rows = df.loc[(df['Brand'] == 'Mercedes - Benz') & (df['Tranmission'] == 'Otomatik')]

# Compute the average price of the selected rows
avg_price = selected_rows['Price'].mean()

print(avg_price)


### 8. Find the number of cars whose price is less than 100K TL, between 100K(exclusive) and 200k (inclusive) and larger than 200k(exclusive). The output should be dictionary.


In [ ]:
# Create a new column with a categorical value based on the price
df['Price Range'] = df['Price'].apply(
    lambda x: '0-100' if x < 100000 else ('100 - 200' if x <= 200000 else '200 and over'))

# Count the number of cars in each category
counts = df['Price Range'].value_counts()

# Convert the Series to a dictionary
counts_dict = counts.to_dict()

print(counts_dict)


### 9. Write a function which generates the lists above automatically.

In [ ]:
def count_cars_by_price_range(df, ranges):
    # Initialize a dictionary to store the counts for each price range
    counts_dict = {f"0-{ranges[0]}": 0}
    for i in range(len(ranges)-1):
        counts_dict[f"{ranges[i]}-{ranges[i+1]}"] = 0
    counts_dict[f"{ranges[-1]} and over"] = 0

    # Iterate over the prices and increment the appropriate count
    for price in df['Price']:
        if price < ranges[0] * 1000:
            counts_dict[f"0-{ranges[0]}"] += 1
        elif price >= ranges[-1] * 1000:
            counts_dict[f"{ranges[-1]} and over"] += 1
        else:
            for i in range(len(ranges)-1):
                if ranges[i] * 1000 <= price < ranges[i+1] * 1000:
                    counts_dict[f"{ranges[i]}-{ranges[i+1]}"] += 1
                    break

    return counts_dict


In [ ]:
# Test the function
counts_dict = count_cars_by_price_range(df, [50, 100, 150, 200, 250])
print(counts_dict)


### 10. Find the number of cars that has an automatic transmission whose price is less than 100K TL, between 100K(exclusive) and 200k (inclusive) and larger than 200k(exclusive).


In [ ]:
# Count the number of cars in each price range
counts_dict = count_cars_by_price_range(df, [100, 200])

# Find the number of cars whose price is less than 100K TL
less_than_100K = counts_dict["0-100"]

# Find the number of cars whose price is between 100K(exclusive) and 200k (inclusive)
between_100K_and_200K = counts_dict["100-200"]

# Find the number of cars whose price is larger than 200k(exclusive)
over_200K = counts_dict["200 and over"]

# Print the counts
print(f"Less than 100: {less_than_100K}")
print(f"Between 100 and 200: {between_100K_and_200K}")
print(f"Larger than 200: {over_200K}")


### 11. For every brand, find the difference for the average prices of automatic transmission and manual transmission and return it as a dictionary.


In [ ]:
# Group the DataFrame by the 'Brand' column and the 'Transmission' column
df_grouped = df.groupby(['Brand', 'Tranmission'])

# Calculate the average price for each group
df_mean = df_grouped['Price'].mean()

# Convert the resulting Series to a DataFrame
df_mean = df_mean.reset_index()

# Pivot the DataFrame to create columns for each brand and transmission type
df_pivot = df_mean.pivot(index='Brand', columns='Tranmission', values='Price')

# Calculate the difference between the average prices for automatic and manual transmission for each brand
df_diff = df_pivot['Otomatik'].subtract(df_pivot['Manuel'])

# Convert the resulting Series to a dictionary
diff_dict = df_diff.to_dict()

print(diff_dict)


### 12. Create a new dataframe which is ordered in terms of the prices of the cars.


In [ ]:
pd_sorted_by_price = df.sort_values('Price', ascending=False)

# Convert the DataFrame to a excel file
pd_sorted_by_price.to_excel('cars_sorted_by_price.xlsx', index=False)


### 13. Create a new dataframe which is ordered in terms of the cities first and then prices of the cars.


In [ ]:
pd_sorted_by_city_and_price = df.sort_values(['City', 'Price'], ascending=[True, False])

# Convert the DataFrame to a excel file
pd_sorted_by_city_and_price.to_excel('cars_sorted_by_city_and_price.xlsx', index=False)


### 14. Divide the price of the cars by its power (Cost/power) and add it as a new column to the datafame.


In [ ]:
# Create a new column with the price per horsepower and add it to the DataFrame
df['CostPerPower'] = df['Price'] / df['Power']

# Convert the DataFrame to a excel file
df.to_excel('cars_with_cost_per_power.xlsx', index=False)


### 15. How many different series each brand have? The output should be a dictionary.


In [ ]:
# Group the DataFrame by the 'Brand' column and count how many series each brand has
df_grouped = df.groupby('Brand')['Series'].nunique()

# count how many series each brand has and create a dictionary
brand_series_count_dict = {}

for brand, count in df_grouped.items():
    brand_series_count_dict[brand] = count

print(brand_series_count_dict)


### 16. Write a function called analyze_car_data.


In [ ]:
def analyze_car_data(count_type, property_type):
    if count_type == "Price":
        return df.groupby(property_type)["Price"].mean().to_dict()
    elif count_type == "Count":
        return df[property_type].value_counts().to_dict()
    elif count_type == "Power":
        return df.groupby(property_type)["Power"].mean().to_dict()
    else:
        raise ValueError("Invalid count type. Choose from 'Price', 'Count', or 'Power'.")


In [ ]:
# If you call the function as follows then it will return the average car price of each city.
analyze_car_data("Price", "City")


In [ ]:
# For example, if you call the function as follows then it will return the number of cars for each transmission type.
analyze_car_data("Count", "Tranmission")


In [ ]:
# If you call the function as follows then it will return the average power of cars for each Brand.
analyze_car_data("Power", "Brand")
